Plot Antarctic Tide Range
=========================

Demonstrates plotting the tidal amplitudes around Antarctica

OTIS format tidal solutions provided by Ohio State University and ESR  
- http://volkov.oce.orst.edu/tides/region.html  
- https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/
- ftp://ftp.esr.org/pub/datasets/tmd/  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  

Finite Element Solution (FES) provided by AVISO  
- https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html
        
#### Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](http://matplotlib.org/)  
 - [cartopy: Python package designed for geospatial data processing](https://scitools.org.uk/cartopy/docs/latest/)  

#### Program Dependencies

- `convert_ll_xy.py`: convert lat/lon points to and from projected coordinates  
- `model.py`: retrieves tide model parameters for named tide models  
- `read_tide_model.py`: extract tidal harmonic constants from OTIS tide models  
- `read_netcdf_model.py`: extract tidal harmonic constants from netcdf models  
- `read_GOT_model.py`: extract tidal harmonic constants from GSFC GOT models  
- `read_FES_model.py`: extract tidal harmonic constants from FES tide models  

This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.  
The widgets can be installed as described below.  
```
pip3 install --user ipywidgets
jupyter nbextension install --user --py widgetsnbextension
jupyter nbextension enable --user --py widgetsnbextension
jupyter-notebook
```

#### Load modules

In [ ]:
import os
import pyproj
import numpy as np
import matplotlib
matplotlib.rcParams['axes.linewidth'] = 2.0
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import ipywidgets as widgets
from IPython.display import HTML

import pyTMD.model
from pyTMD.read_tide_model import extract_tidal_constants
from pyTMD.read_netcdf_model import extract_netcdf_constants
from pyTMD.read_GOT_model import extract_GOT_constants
from pyTMD.read_FES_model import extract_FES_constants
# autoreload
%load_ext autoreload
%autoreload 2

#### Set parameters  for program

- Model directory  
- Tide model  

In [ ]:
# set the directory with tide models
dirText = widgets.Text(
    value=os.getcwd(),
    description='Directory:',
    disabled=False
)

# dropdown menu for setting tide model
model_list = ['CATS0201','CATS2008','TPXO9-atlas','TPXO9-atlas-v2',
    'TPXO9-atlas-v3','TPXO9-atlas-v4','TPXO9-atlas-v5','TPXO9.1',
    'TPXO8-atlas','TPXO7.2','GOT4.7','GOT4.8','GOT4.10','FES2014',
    'EOT20']
modelDropdown = widgets.Dropdown(
    options=model_list,
    value='CATS2008',
    description='Model:',
    disabled=False,
)

# dropdown menu for setting ATLAS format model
atlas_list = ['OTIS','netcdf']
atlasDropdown = widgets.Dropdown(
    options=atlas_list,
    value='netcdf',
    description='ATLAS:',
    disabled=False,
)

# checkbox for setting if tide files are compressed
compressCheckBox = widgets.Checkbox(
    value=True,
    description='Compressed?',
    disabled=False,
)

# display widgets for setting directory and model
widgets.VBox([
    dirText,
    modelDropdown,
    atlasDropdown,
    compressCheckBox
])

#### Setup tide model parameters

In [ ]:
# get model parameters
model = pyTMD.model(dirText.value, format=atlasDropdown.value,
    compressed=compressCheckBox.value).elevation(modelDropdown.value)

#### Setup coordinates for calculating tides

In [ ]:
# create an image around Antarctica
xlimits = [-560.*5e3,560.*5e3]
ylimits = [-560.*5e3,560.*5e3]
spacing = [5e3,-5e3]
# x and y coordinates
x = np.arange(xlimits[0],xlimits[1]+spacing[0],spacing[0])
y = np.arange(ylimits[1],ylimits[0]+spacing[1],spacing[1])
xgrid,ygrid = np.meshgrid(x,y)
# x and y dimensions
nx = int((xlimits[1]-xlimits[0])/spacing[0])+1
ny = int((ylimits[0]-ylimits[1])/spacing[1])+1
# convert image coordinates from polar stereographic to latitude/longitude
crs1 = pyproj.CRS.from_string("epsg:{0:d}".format(3031))
crs2 = pyproj.CRS.from_string("epsg:{0:d}".format(4326))
transformer = pyproj.Transformer.from_crs(crs1, crs2, always_xy=True)
lon,lat = transformer.transform(xgrid.flatten(), ygrid.flatten())

#### Infer minor amplitudes from the major constituents

In [ ]:
def infer_minor_amplitudes(zmajor,constituents):
    # number of constituents
    npts,nc = np.shape(zmajor)    
    cindex = ['q1','o1','p1','k1','n2','m2','s2','k2']
    # re-order zmajor to correspond to cindex
    z8 = np.ma.zeros((npts,8))
    ni = 0
    for i,c in enumerate(cindex):
        j = [j for j,val in enumerate(constituents) if val == c]
        if j:
            j1, = j
            z8[:,i] = zmajor[:,j1]
            ni += 1
    # list of minor constituents
    minor = ['2q1','sigma1','rho1','m1','m1','chi1','pi1','phi1','theta1',
        'j1','oo1','2n2','mu2','nu2','lambda2','l2','l2','t2']
    # only add minor constituents that are not on the list of major values
    minor_flag = [m not in constituents for m in minor]
    # estimate minor constituents
    zmin = np.zeros((npts,18))
    zmin[:,0] = 0.263*z8[:,0] - 0.0252*z8[:,1]# 2Q1
    zmin[:,1] = 0.297*z8[:,0] - 0.0264*z8[:,1]# sigma1
    zmin[:,2] = 0.164*z8[:,0] + 0.0048*z8[:,1]# rho1
    zmin[:,3] = 0.0140*z8[:,1] + 0.0101*z8[:,3]# M1
    zmin[:,4] = 0.0389*z8[:,1] + 0.0282*z8[:,3]# M1
    zmin[:,5] = 0.0064*z8[:,1] + 0.0060*z8[:,3]# chi1
    zmin[:,6] = 0.0030*z8[:,1] + 0.0171*z8[:,3]# pi1
    zmin[:,7] = -0.0015*z8[:,1] + 0.0152*z8[:,3]# phi1
    zmin[:,8] = -0.0065*z8[:,1] + 0.0155*z8[:,3]# theta1
    zmin[:,9] = -0.0389*z8[:,1] + 0.0836*z8[:,3]# J1
    zmin[:,10] = -0.0431*z8[:,1] + 0.0613*z8[:,3]# OO1
    zmin[:,11] = 0.264*z8[:,4] - 0.0253*z8[:,5]# 2N2
    zmin[:,12] = 0.298*z8[:,4] - 0.0264*z8[:,5]# mu2
    zmin[:,13] = 0.165*z8[:,4] + 0.00487*z8[:,5]# nu2
    zmin[:,14] = 0.0040*z8[:,5] + 0.0074*z8[:,6]# lambda2
    zmin[:,15] = 0.0131*z8[:,5] + 0.0326*z8[:,6]# L2
    zmin[:,16] = 0.0033*z8[:,5] + 0.0082*z8[:,6]# L2
    zmin[:,17] = 0.0585*z8[:,6]#-- t2
    # only add minor constituents that are not major
    return np.where(minor_flag, np.abs(zmin), 0.0)

#### Calculate tide map

In [ ]:
# read tidal constants and interpolate to grid points
if model.format in ('OTIS','ATLAS'):
    amp,ph,D,c = extract_tidal_constants(lon, lat, model.grid_file,
        model.model_file, model.projection, TYPE=model.type,
        METHOD='spline', GRID=model.format)
elif (model.format == 'netcdf'):
    amp,ph,D,c = extract_netcdf_constants(lon, lat, model.grid_file,
        model.model_file, TYPE=model.type, METHOD='spline',
        SCALE=model.scale, GZIP=model.compressed)
elif (model.format == 'GOT'):
    amp,ph,c = extract_GOT_constants(lon, lat, model.model_file,
        METHOD='spline', SCALE=model.scale, GZIP=model.compressed)
elif (model.format == 'FES'):
    amp,ph = extract_FES_constants(lon, lat, model.model_file,
        TYPE=model.type, VERSION=model.version, METHOD='spline',
        SCALE=model.scale, GZIP=model.compressed)
    c = model.constituents

# calculate minor constituent amplitudes
minor_amp = infer_minor_amplitudes(amp,c)
# calculate sum of major and minor amplitudes
tide_range = np.sum(amp,axis=1) + np.sum(minor_amp,axis=1)
# convert from meters to centimeters
tide_cm = 100.0*np.reshape(tide_range,(ny,nx))

#### Create plot of tidal range

In [ ]:
# plot Antarctic tide range
projection = ccrs.Stereographic(central_longitude=0.0,
    central_latitude=-90,true_scale_latitude=-71.0)
fig, ax = plt.subplots(num=1, figsize=(9,8),
    subplot_kw=dict(projection=projection))
# plot tide height
vmin,vmax = (0, np.max(tide_cm))
extent = (xlimits[0],xlimits[1],ylimits[0],ylimits[1])
im = ax.imshow(tide_cm, interpolation='nearest',
    vmin=vmin, vmax=vmax, transform=projection,
    extent=extent, origin='upper')
# add high resolution cartopy coastlines
ax.coastlines('10m')

# Add colorbar and adjust size
# pad = distance from main plot axis
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax, pad=0.025, extend='max',
    extendfrac=0.0375, shrink=0.85, aspect=22.5, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_ylabel('{0} Tide Range'.format(model.name), fontsize=13)
cbar.ax.set_xlabel('cm', fontsize=13)
cbar.ax.xaxis.set_label_coords(0.50, 1.04)
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=18,
    labelsize=13, direction='in')

# set x and y limits
ax.set_xlim(xlimits)
ax.set_ylim(ylimits)

# stronger linewidth on frame
ax.spines['geo'].set_linewidth(2.0)
ax.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
# show the plot
plt.show()